In [9]:
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import Image
import math

In [10]:
def get_the_info(link):
    """
    Recives the requests.get(url) and returns the ranks, titles, scores, formats, date and images(?)
    of every anime in the page. Images are a work in progress, maybe in the future
    """
    webcode=BeautifulSoup(link.content,'lxml')
    rankscode=webcode.select('td[class="rank ac"]>span')
    ranks=[i.text for i in rankscode]
    imagescode=webcode.select('td[class="title al va-t word-break"]>a>img')
    images=[Image(url=i['data-src']) for i in imagescode]
    #df['Images']=[display(i) for i in images]
    titlescode=webcode.select('div[class="di-ib clearfix"]>h3>a')
    titles=[i.text for i in titlescode]
    scorescode=webcode.select('td[class="score ac fs14"]>div>span')
    scores=[i.text for i in scorescode]
    detailscode=webcode.select('div[class="detail"]>div[class="information di-ib mt4"]')
    formats=[i.text.split('        ')[1].strip() for i in detailscode]
    date=[i.text.split('        ')[2].strip() for i in detailscode]
    return ranks,titles,scores,formats,date,images

In [11]:
def get_more_info(link):
    """
    Recives the requests.get(url),takes all the links in the page and returns the links,
    studio, genres, source, rating, recommendation of every anime in the page
    """
    webcode=BeautifulSoup(link.content,'lxml')
    linkscode=webcode.select('h3[class="hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3"]>a')
    links=[i['href'] for i in linkscode]
    producers=[]
    studio=[]
    genres=[]
    indvgenres=[]
    source=[]
    rating=[]
    recommendation=[]
    for i in links:
        time.sleep(2)
        sublink=requests.get(i)
        subwebcode=BeautifulSoup(sublink.content,'lxml')
        studiocode=subwebcode.select('div[class="spaceit_pad"]>a')
        for i in studiocode:
            if i['href'][:15]==('/anime/producer'):
                producers.append(i.text)
        studio.append(producers[-1])
        genrescode=subwebcode.select('div[class="spaceit_pad"]>a')
        for i in genrescode:
            if i['href'][:12]==('/anime/genre'):
                indvgenres.append(i.text)
        genres.append(indvgenres)
        indvgenres=[]
        sourcecode=subwebcode.select('div[class="spaceit_pad"]')
        for i in sourcecode:
            if i.text[:7]=='\nSource':
                source.append(i.text.split('\n')[2].strip())
        for i in sourcecode:
            if i.text[:7]=='\nRating':
                rating.append(i.text.split('\n')[2].strip())
        recommendationcode=subwebcode.select('a[class="link bg-center"]>span')
        recs=[i.text for i in recommendationcode]
        recommendation.append(recs[0])
    return links,studio,genres,source,rating,recommendation

In [12]:
def put_it_in(info,more_info):
    """
    Recives the info regarding the get_the_info and get_more_info functions and joins them 
    in a pandas DataFrame
    """
    df=pd.DataFrame()
    df['Rank']=info[0]
    df['Title']=info[1]
    df['Score']=info[2]
    df['Format']=info[3]
    df['Date']=info[4]
    df['Studio']=more_info[1]
    df['Genres']=more_info[2]
    df['Source']=more_info[3]
    df['Rating']=more_info[4]
    df['Recommendation']=more_info[5]
    df['Links']=more_info[0]
    return df

In [44]:
def popular_animes(number):
    """
    Recives the number of animes you want to see and returns a pandas DataFrame and a .csv file with
    all the anime you requested
    """
    pages = 50 * math.floor((number-1)/50)
    url='https://myanimelist.net/topanime.php?type=bypopularity'
    link=requests.get(url)
    url2='&limit='
    animes=put_it_in(get_the_info(link),get_more_info(link))
    global anilinks
    if pages==0:
        anilinks=animes.pop('Links')
        anilinks=anilinks.reset_index(drop=True)
        animes=animes[:number]
        animes=animes.reset_index(drop=True)
        animes.to_csv('popular_anime.csv',index=False,sep=',')
        return animes
    else:
        for i in range(50,number,50):
            combinedurl=url+url2+str(i)
            combinedlink=requests.get(combinedurl)
            more_animes=put_it_in(get_the_info(combinedlink),get_more_info(combinedlink))
            animes=pd.concat([animes,more_animes])
        anilinks=animes.pop('Links')
        anilinks=anilinks.reset_index(drop=True)
        animes=animes[:number]
        animes=animes.reset_index(drop=True)
        animes.to_csv('popular_anime.csv',index=False,sep=',')
        return animes

In [45]:
test=popular_animes(100)

In [46]:
test

,Rank,Title,Score,Format,Date,Studio,Genres,Source,Rating,Recommendation
0,1,Death Note,8.63,TV (37 eps),Oct 2006 - Jun 2007,Madhouse,"[Mystery, Supernatural, Suspense, Police, Psyc...",Manga,R - 17+ (violence & profanity),Code Geass: Hangyaku no Lelouch
1,2,Shingeki no Kyojin,8.51,TV (25 eps),Apr 2013 - Sep 2013,Wit Studio,"[Action, Drama, Fantasy, Mystery, Military, Su...",Manga,R - 17+ (violence & profanity),Koutetsujou no Kabaneri
2,3,Fullmetal Alchemist: Brotherhood,9.16,TV (64 eps),Apr 2009 - Jul 2010,Bones,"[Action, Adventure, Comedy, Drama, Fantasy, Mi...",Manga,R - 17+ (violence & profanity),Hunter x Hunter (2011)
3,4,Sword Art Online,7.21,TV (25 eps),Jul 2012 - Dec 2012,A-1 Pictures,"[Action, Adventure, Fantasy, Romance, Game]",Light novel,PG-13 - Teens 13 or older,Log Horizon
4,5,One Punch Man,8.53,TV (12 eps),Oct 2015 - Dec 2015,Madhouse,"[Action, Comedy, Sci-Fi, Supernatural, Parody,...",Web manga,R - 17+ (violence & profanity),Mob Psycho 100
...,...,...,...,...,...,...,...,...,...,...
95,96,Made in Abyss,8.70,TV (13 eps),Jul 2017 - Sep 2017,Kinema Citrus,"[Adventure, Drama, Fantasy, Mystery, Sci-Fi]",Web manga,R - 17+ (violence & profanity),Yakusoku no Neverland
96,97,Clannad: After Story,8.95,TV (24 eps),Oct 2008 - Mar 2009,Kyoto Animation,"[Comedy, Drama, Romance, Slice of Life, Supern...",Visual novel,PG-13 - Teens 13 or older,Shigatsu wa Kimi no Uso
97,98,Howl no Ugoku Shiro,8.66,Movie (1 eps),Nov 2004 - Nov 2004,Studio Ghibli,"[Adventure, Drama, Fantasy, Romance]",Novel,G - All Ages,Sen to Chihiro no Kamikakushi
98,99,Mononoke Hime,8.69,Movie (1 eps),Jul 1997 - Jul 1997,Studio Ghibli,"[Action, Adventure, Fantasy]",Original,PG-13 - Teens 13 or older,Kaze no Tani no Nausicaä
